In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession 
from math import ceil,fsum
spark = SparkSession.builder.master("local[4]").appName("session1").getOrCreate()
sc = spark.sparkContext
product = sc.textFile("products.txt")

21/11/04 14:29:56 WARN Utils: Your hostname, pamousso-Latitude-E5530-non-vPro resolves to a loopback address: 127.0.1.1; using 192.168.1.13 instead (on interface wlp2s0)
21/11/04 14:29:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/04 14:29:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/04 14:29:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
"""
Start of pratical sessions part I
"""
def maxPrice(a, b):
    if a>b:
        return a
    else:
        return b

def reduce_count(a, b):
    return a+b

def avg(a,b):
    return 0.5*(a+b)


In [4]:
#prod_header = ["tid", "name", "price", "category"]
prod = product.map(lambda line: (line.split())).map(lambda line:(int(line[0]) , line[1], float(line[2]), line[3]))
prod.collect()

[(1, 'apple', 3.3, 'fruit'),
 (2, 'orange', 6.5, 'fruit'),
 (3, 'beer', 10.0, 'beverage'),
 (4, 'wine', 15.0, 'beverage'),
 (5, 'TAOCP1', 20.9, 'book'),
 (6, 'TAOCP2', 5.2, 'book'),
 (7, 'film1', 30.0, 'film'),
 (8, 'film2', 20.4, 'film'),
 (9, 'film3', 33.0, 'film'),
 (10, 'film4', 42.9, 'film'),
 (11, 'film5', 13.3, 'film'),
 (12, 'milk', 7.6, 'beverage'),
 (13, 'banana', 5.3, 'fruit'),
 (14, 'grapes', 7.5, 'fruit'),
 (15, 'soda', 4.0, 'beverage'),
 (16, 'water', 2.0, 'beverage'),
 (17, 's_water', 3.5, 'beverage'),
 (18, 'grappe', 8.2, 'fruit'),
 (19, 'pear', 7.0, 'fruit'),
 (20, 'strawberry', 9.0, 'fruit'),
 (21, 'plum', 4.4, 'fruit'),
 (22, 'mango', 17.5, 'fruit')]

In [5]:
maxpricebytcat = prod.map(lambda line : (line[3], line[2])).reduceByKey(maxPrice)
print("Max price by categorie")
maxpricebytcat.collect()


Max price by categorie


[('beverage', 15.0), ('book', 20.9), ('fruit', 17.5), ('film', 42.9)]

In [6]:
price = prod.filter(lambda line : line[2]>20)
price.collect()

[(5, 'TAOCP1', 20.9, 'book'),
 (7, 'film1', 30.0, 'film'),
 (8, 'film2', 20.4, 'film'),
 (9, 'film3', 33.0, 'film'),
 (10, 'film4', 42.9, 'film')]

In [7]:
countbycat = prod.map(lambda line :(line[3], 1)).reduceByKey(reduce_count)
print("Quantity of product by categorie")
countbycat.collect()

Quantity of product by categorie


[('beverage', 6), ('book', 2), ('fruit', 9), ('film', 5)]

In [8]:
# 7)
avgbycat = prod.map(lambda line: (line[3],line[2])).reduceByKey(avg)
avgbycat.collect()

[('beverage', 6.65),
 ('book', 13.049999999999999),
 ('fruit', 8.384375),
 ('film', 24.65)]

In [9]:
avgbycat = prod.map(lambda line: (line[3],line[2])).sortByKey()#.reduceByKey(avg)
avgbycat.collect()

[('beverage', 10.0),
 ('beverage', 15.0),
 ('beverage', 7.6),
 ('beverage', 4.0),
 ('beverage', 2.0),
 ('beverage', 3.5),
 ('book', 20.9),
 ('book', 5.2),
 ('film', 30.0),
 ('film', 20.4),
 ('film', 33.0),
 ('film', 42.9),
 ('film', 13.3),
 ('fruit', 3.3),
 ('fruit', 6.5),
 ('fruit', 5.3),
 ('fruit', 7.5),
 ('fruit', 8.2),
 ('fruit', 7.0),
 ('fruit', 9.0),
 ('fruit', 4.4),
 ('fruit', 17.5)]

In [10]:
# 8)
pricebycat = prod.map(lambda line :(line[3], (line[2],1))).sortByKey(False)
pricebycat = pricebycat.reduceByKey(lambda x, y: (x[0]+y[0], x[1] + y[1]))
avg_price = pricebycat.map(lambda line : {line[0]: (line[1][0]/line[1][1])})
avg_price.collect()
#pribycat = pricebycat.map(lambda line:(line, countbycat.collect()[0]))
#pribycat.collect()

[{'book': 13.049999999999999},
 {'beverage': 7.016666666666667},
 {'fruit': 7.633333333333332},
 {'film': 27.920000000000005}]

In [11]:
################### Part III ###################
file_A= sc.textFile("A.txt")
file_B= sc.textFile("B.txt")
file_A = file_A.map(lambda l: int(l))
file_B = file_B.map(lambda l: int(l))
file_A.collect()

[1, 1, 2, 2, 4, 6, 8, 10, 10, 10, 10, 12, 14]

In [12]:
file_B.collect()

[1, 1, 1, 1, 1, 3, 5, 7, 9, 10, 10, 10, 12, 14, 16, 18, 20]

In [13]:

file_union = file_A.union(file_B).map(lambda line:int(line))
file_union.collect()


[1,
 1,
 2,
 2,
 4,
 6,
 8,
 10,
 10,
 10,
 10,
 12,
 14,
 1,
 1,
 1,
 1,
 1,
 3,
 5,
 7,
 9,
 10,
 10,
 10,
 12,
 14,
 16,
 18,
 20]

In [14]:
file_intersection = file_A.intersection(file_B).map(lambda line:int(line))
file_intersection.collect()

[12, 1, 10, 14]

In [15]:
file_union_bis = file_A.map(lambda line: (line, 1)).union(file_B.map(lambda line: (line, 1)))
file_union_bis = file_union_bis.map(lambda line: [line[0],1]).reduceByKey(lambda a,b : a+b)
file_union_bis.sortByKey().collect()


15


[(1, 7),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 7),
 (12, 2),
 (14, 2),
 (16, 1),
 (18, 1),
 (20, 1)]

In [39]:
file_A_reduce = file_A.map(lambda line: (line, 1)).reduceByKey(lambda a,b : a+b)
file_B_reduce = file_B.map(lambda line: (line, 1)).reduceByKey(lambda a,b : a+b)

def clean(line):
    if(line[1][0] is None):
        line = (line[0], (0, line[1][1]))
    if(line[1][1] is None):
        line = (line[0], (line[1][0],0))
    return line

def minim(a,b):
    return min(a,b)


In [37]:
file_join = file_A_reduce.rightOuterJoin(file_B_reduce).union(file_A_reduce.leftOuterJoin(file_B_reduce))
file_join = file_join.map(clean).distinct()#.reduceByKey(lambda a,b: a) # Enlévé les doublons
file_join.collect()

[(12, (1, 1)),
 (14, (1, 1)),
 (2, (2, 0)),
 (1, (2, 5)),
 (3, (0, 1)),
 (5, (0, 1)),
 (7, (0, 1)),
 (9, (0, 1)),
 (10, (4, 3)),
 (16, (0, 1)),
 (18, (0, 1)),
 (20, (0, 1)),
 (4, (1, 0)),
 (6, (1, 0)),
 (8, (1, 0))]

In [40]:
def nTimes(line):
    times = []
    for i in range(line[1]):
        times.append(line[0])
    return times

file_intersect = file_join.map(lambda line: (line[0], minim(line[1][0], line[1][1])))#.reduceByKey(minim)
file_intersect = file_intersect.filter(lambda line: line[1]!=0).flatMap(nTimes)
file_intersect.collect()

[12, 14, 1, 1, 10, 10, 10]

In [49]:
def maxim(a, b):
    print(max(0,abs(a-b)))
    return max(0,abs(a-b))

def symDif(a,b):
    print(abs(max(a,b)-min(a,b)))
    #print("{} - {}".format(max(a,b),min(a,b)))
    return abs(max(a,b)-min(a,b))

In [50]:
file_intersect = file_join.map(lambda line: (line[0], maxim(line[1][0], line [1][1])))#file_intersect = file_intersect.filter(lambda line: line[1]!=0)
file_intersect = file_intersect.filter(lambda line: line[1]!=0).flatMap(nTimes)
file_intersect.collect()

0
0
2
3
1
1
1
1
1
1
1
1
1
1
1


[2, 2, 1, 1, 1, 3, 5, 7, 9, 10, 16, 18, 20, 4, 6, 8]

In [51]:
file_intersect = file_join.map(lambda line: (line[0], symDif(line[1][0], line [1][1])))#file_intersect = file_intersect.filter(lambda line: line[1]!=0)
file_intersect = file_intersect.filter(lambda line: line[1]!=0).flatMap(nTimes)
file_intersect.collect()

1
1
1
1
1
1
1
0
0
2
3
1
1
1
1


[2, 2, 1, 1, 1, 3, 5, 7, 9, 10, 16, 18, 20, 4, 6, 8]